### Install Dependencies

In [ ]:
!pip3 install git+https://github.com/mit-nlp/MITIE.git
!pip3 install rasa_nlu[mitie]
!pip3 install rasa_core==0.9.6
!wget https://github.com/mit-nlp/MITIE/releases/download/v0.4/MITIE-models-v0.2.tar.bz2
!tar jxf MITIE-models-v0.2.tar.bz2

### 1.a. Configure Interpreter Pipeline

In [ ]:
with open('interpreter.yml','w') as f:
  f.write('''
  language: "en"

  pipeline:
  - name: "nlp_mitie"
    model: "MITIE-models/english/total_word_feature_extractor.dat"
  - name: "tokenizer_mitie"
  - name: "ner_mitie"
  - name: "ner_synonyms"
  - name: "intent_entity_featurizer_regex"
  - name: "intent_featurizer_mitie"
  - name: "intent_classifier_sklearn"
  ''')

### 1.b. Prepare Interpreter Training Data

Format:

''##intent: intent_label"

"-example utterance for intent "

"-another example utterance with an [entity] (entity_label)"

In [ ]:
with open('interpreter_dataset.md','w') as f:
  f.write('''
  

  ## intent:intent_product
  - You have been using our [prepaid connection](product)
  - using our [prepaid connection](product)
  - you are part of [prepaid connection](product)
  - [postpaid](product)
  - [prepaid](product)

  
  ## intent:intent_intro
  - hello my name is [MIKE](name) from [AT&T incorporation](company)
  - hi this is [MIKE](name) from [goog](company)
  - My name is [Alice](name) from [AT&T](company)
  - I'm [Lucy](name)
  - I am [Richard](name)
  - Call me [Sally](name)

  ##intent:intent_onboard
  -good [sign me](onboard) up
  -fantastic [sign me](onboard) up
  -[sign me](onboard) up
  -[signup](onboard)


  ## intent:intent_ask_purpose
  - yes How can I [help you](purpose)? 
  - yeah How can I [help you](purpose)? 
  - sure what do you [want](purpose)
  - [help](purpose)
  - what's the [catch](purpose)
  - what is the [plan](purpose)

  ##regex:purpose
  -[a-z]{4,10}

  ##intent:intent_affirmative
  - that's [right](affirmation)
  - yep that's [right](affirmation)
  - I would [love](affirmation)

  ##regex:affirmation
  -[a-z]{4,6}

  ##intent:intent_quantity
  - a couple[quantity] of years
  - for ten[quantity] years
  - about 2[quantity] years
  - be a monthly[quantity] rental
  - a quarterly[quantity] rental
  - free unlimited[quantity] voice calls
  - upto 100[quantity] calls
 
  ''')

### 1.c. Train the Interpreter

In [ ]:
!python3 -m rasa_nlu.train -c interpreter.yml -d interpreter_dataset.md -o models --fixed_model_name interpreter --verbose

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-06-22 12:23:46 WARNING  rasa_nlu.registry  - DEPRECATION warning: your nlu config file contains old style component name `nlp_mitie`, you should change it to its class name: `MitieNLP`.
2022-06-22 12:23:46 WARNING  rasa_nlu.registry  - DEPRECATION warning: your nlu config file contains old style component name `tokenizer_mitie`, you should change it to its class name: `MitieTokenizer`.
2022-06-22 12:23:46 WARNING  rasa_nlu.registry  - DEPRECATION warning: your nlu config file contains old style component name `ner_mitie`, you should change it to its class name: `MitieEntityExtractor`.
2022-06-22 12:23:46 WARNING  rasa_nlu.registry  - DEPRECATION warning: your nlu config file contains old style component name `ner_synonyms`, you should change it to

### 1.d. Test the Interpreter

In [ ]:
!pip install --upgrade rasa-core

In [ ]:
from rasa_core.interpreter import RasaNLUInterpreter
interpreter = RasaNLUInterpreter('models/default/interpreter')

In [ ]:
!zip -r ./models.zip ./models/

  adding: models/ (stored 0%)
  adding: models/default/ (stored 0%)
  adding: models/default/interpreter/ (stored 0%)
  adding: models/default/interpreter/component_6_SklearnIntentClassifier.pkl (deflated 37%)
  adding: models/default/interpreter/metadata.json (deflated 71%)
  adding: models/default/interpreter/training_data.json (deflated 88%)
  adding: models/default/interpreter/component_2_MitieEntityExtractor.dat (deflated 99%)
  adding: models/default/interpreter/component_4_RegexFeaturizer.pkl (stored 0%)


In [ ]:
with open("talk (1).txt", "r") as fic:
    content = fic.read().replace('\n', '#')
sentences = list(map(str.strip, content.split("#")))

In [136]:
def listToString(s): 
    
    # initialize an empty string
    str1 = " " 
    
    # return string  
    return (str1.join(s))
actual_text=listToString(sentences)
print(listToString(sentences)) 

Hello hi Nancy, this is Mike from AT&T incorporation Yes, How can I help you? Nancy,You have been using our prepaid connection for a couple of years now, right? Here that's right How would you like a postpaid connection that allows you to make free unlimited voice calls to three AT&T numbers? I would love that, but what's the catch? There's no catch There will be a monthly rental which you will have to pay like any other postpaid connection Fantastic sign me up


In [ ]:
interpreter.parse(sentences[0])

{'entities': [{'confidence': None,
   'end': 14,
   'entity': 'name',
   'extractor': 'MitieEntityExtractor',
   'start': 9,
   'value': 'Nancy'},
  {'confidence': None,
   'end': 28,
   'entity': 'name',
   'extractor': 'MitieEntityExtractor',
   'start': 24,
   'value': 'Mike'},
  {'confidence': None,
   'end': 52,
   'entity': 'company',
   'extractor': 'MitieEntityExtractor',
   'start': 34,
   'value': 'AT&T incorporation'}],
 'intent': {'confidence': 0.5589540432470059, 'name': 'intent_intro'},
 'intent_ranking': [{'confidence': 0.5589540432470059, 'name': 'intent_intro'},
  {'confidence': 0.10325674894998398, 'name': 'intent_ask_purpose'},
  {'confidence': 0.09807627859636418, 'name': 'intent_affirmative'},
  {'confidence': 0.09190429638232776, 'name': 'intent_quantity'},
  {'confidence': 0.08988053163820803, 'name': 'intent_product'},
  {'confidence': 0.05792810118611027, 'name': 'intent_onboard'}],
 'model': 'current',
 'project': 'default',
 'text': 'Hello hi Nancy, this is M

In [ ]:
x=[]
for y in range(0,len(sentences)):
  print(interpreter.parse(sentences[y]))
  x.append(interpreter.parse(sentences[y]))

{'intent': {'name': 'intent_intro', 'confidence': 0.593252456288569}, 'entities': [{'entity': 'name', 'value': 'Nancy', 'start': 9, 'end': 14, 'confidence': None, 'extractor': 'MitieEntityExtractor'}, {'entity': 'name', 'value': 'Mike', 'start': 24, 'end': 28, 'confidence': None, 'extractor': 'MitieEntityExtractor'}, {'entity': 'company', 'value': 'AT&T incorporation', 'start': 34, 'end': 52, 'confidence': None, 'extractor': 'MitieEntityExtractor'}], 'intent_ranking': [{'name': 'intent_intro', 'confidence': 0.593252456288569}, {'name': 'intent_ask_purpose', 'confidence': 0.09150738245731746}, {'name': 'intent_affirmative', 'confidence': 0.09091475364082119}, {'name': 'intent_product', 'confidence': 0.0908441727348538}, {'name': 'intent_quantity', 'confidence': 0.08126135631082544}, {'name': 'intent_onboard', 'confidence': 0.0522198785676128}], 'text': 'Hello hi Nancy, this is Mike from AT&T incorporation', 'model': 'current', 'project': 'default'}
{'intent': {'name': 'intent_ask_purpos

In [128]:
from collections import OrderedDict
new_dict=OrderedDict()
new_dict_deepcopy=OrderedDict()
from copy import deepcopy
fin_list=[]
for num in range(0,len(x)):
  temp_text=x[num]['text']
  ent=[]
  for z in x[num]['entities']:
    #print(z)
    temp_dict={}
    key=z['entity']
    val=z['value']
    temp_dict[key]=val
    ent.append(temp_dict)
    
    #print(z['entity']+": "+z['value'])
  #print("text:",temp_text)
  #print("intent",x[num]['intent']['name'])
  #print("entity:",ent)
  keys=['text','intent','entity']
  values=[temp_text,x[num]['intent']['name'],ent]
  #new_dict=dict(zip(keys,values))
  new_dict['text']=temp_text
  new_dict['intent']=x[num]['intent']['name']
  new_dict['entity']=ent
  
  print(dict(new_dict))
  new_dict_deepcopy = deepcopy(new_dict)
  fin_list.append(dict(new_dict_deepcopy))
  temp_text=[]
  

{'text': 'Hello hi Nancy, this is Mike from AT&T incorporation', 'intent': 'intent_intro', 'entity': [{'name': 'Nancy'}, {'name': 'Mike'}, {'company': 'AT&T incorporation'}]}
{'text': 'Yes, How can I help you?', 'intent': 'intent_ask_purpose', 'entity': [{'purpose': 'help you'}]}
{'text': 'Nancy,You have been using our prepaid connection for a couple of years now, right?', 'intent': 'intent_product', 'entity': [{'name': 'Nancy'}, {'product': 'prepaid connection'}, {'affirmation': 'right'}]}
{'text': "Here that's right", 'intent': 'intent_affirmative', 'entity': [{'affirmation': 'right'}]}
{'text': 'How would you like a postpaid connection that allows you to make free unlimited voice calls to three AT&T numbers?', 'intent': 'intent_ask_purpose', 'entity': [{'affirmation': 'you'}, {'product': 'postpaid connection'}, {'affirmation': 'you'}, {'purpose': 'make free'}, {'company': 'AT&T numbers'}]}
{'text': "I would love that, but what's the catch?", 'intent': 'intent_ask_purpose', 'entity':

In [131]:
print(fin_list)

[{'text': 'Hello hi Nancy, this is Mike from AT&T incorporation', 'intent': 'intent_intro', 'entity': [{'name': 'Nancy'}, {'name': 'Mike'}, {'company': 'AT&T incorporation'}]}, {'text': 'Yes, How can I help you?', 'intent': 'intent_ask_purpose', 'entity': [{'purpose': 'help you'}]}, {'text': 'Nancy,You have been using our prepaid connection for a couple of years now, right?', 'intent': 'intent_product', 'entity': [{'name': 'Nancy'}, {'product': 'prepaid connection'}, {'affirmation': 'right'}]}, {'text': "Here that's right", 'intent': 'intent_affirmative', 'entity': [{'affirmation': 'right'}]}, {'text': 'How would you like a postpaid connection that allows you to make free unlimited voice calls to three AT&T numbers?', 'intent': 'intent_ask_purpose', 'entity': [{'affirmation': 'you'}, {'product': 'postpaid connection'}, {'affirmation': 'you'}, {'purpose': 'make free'}, {'company': 'AT&T numbers'}]}, {'text': "I would love that, but what's the catch?", 'intent': 'intent_ask_purpose', 'en

In [138]:
fin_dict={}

In [140]:
fin_dict['task_1_output']=actual_text
fin_dict['task_3_output']=fin_list

In [165]:
my_dict = {'c':3, 'a':1, 'd':4, 'b':2}

# Sorting dictionary
sorted_dict = fin_dict.keys()
sorted_dict = sorted(sorted_dict)
  
# Printing sorted dictionary
print("Sorted dictionary using sorted() and keys() is : ")
f = open("output_file.txt", "a")
for key in sorted_dict:
    print(key,':', fin_dict[key])
f.write(json.dumps(fin_dict))
f.close()

Sorted dictionary using sorted() and keys() is : 
task_1_output : Hello hi Nancy, this is Mike from AT&T incorporation Yes, How can I help you? Nancy,You have been using our prepaid connection for a couple of years now, right? Here that's right How would you like a postpaid connection that allows you to make free unlimited voice calls to three AT&T numbers? I would love that, but what's the catch? There's no catch There will be a monthly rental which you will have to pay like any other postpaid connection Fantastic sign me up
task_3_output : [{'text': 'Hello hi Nancy, this is Mike from AT&T incorporation', 'intent': 'intent_intro', 'entity': [{'name': 'Nancy'}, {'name': 'Mike'}, {'company': 'AT&T incorporation'}]}, {'text': 'Yes, How can I help you?', 'intent': 'intent_ask_purpose', 'entity': [{'purpose': 'help you'}]}, {'text': 'Nancy,You have been using our prepaid connection for a couple of years now, right?', 'intent': 'intent_product', 'entity': [{'name': 'Nancy'}, {'product': 'pr

In [155]:
f = open("demofile2.txt", "a")
f.write("Now the file has more content!")
f.close()

OrderedDict([('task_1_output',
              "Hello hi Nancy, this is Mike from AT&T incorporation Yes, How can I help you? Nancy,You have been using our prepaid connection for a couple of years now, right? Here that's right How would you like a postpaid connection that allows you to make free unlimited voice calls to three AT&T numbers? I would love that, but what's the catch? There's no catch There will be a monthly rental which you will have to pay like any other postpaid connection Fantastic sign me up"),
             ('task_3_output',
              [{'entity': [{'name': 'Nancy'},
                 {'name': 'Mike'},
                 {'company': 'AT&T incorporation'}],
                'intent': 'intent_intro',
                'text': 'Hello hi Nancy, this is Mike from AT&T incorporation'},
               {'entity': [{'purpose': 'help you'}],
                'intent': 'intent_ask_purpose',
                'text': 'Yes, How can I help you?'},
               {'entity': [{'name': 'Nancy'

In [166]:
import sys
print(sys.modules.keys())

dict_keys(['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimport', '_frozen_importlib_external', '_io', 'marshal', 'posix', 'encodings', 'codecs', '_codecs', 'encodings.aliases', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', 'io', 'abc', '_abc', '_bootlocale', '_locale', 'warnings', 're', 'enum', 'types', '_collections', 'sre_compile', '_sre', 'sre_parse', 'sre_constants', 'functools', '_functools', 'collections', '_collections_abc', 'operator', '_operator', 'keyword', 'heapq', '_heapq', 'itertools', 'reprlib', 'copyreg', 'site', 'os', 'stat', '_stat', 'posixpath', 'genericpath', 'os.path', '_sitebuiltins', 'importlib', 'importlib._bootstrap', 'importlib._bootstrap_external', 'importlib.util', 'importlib.abc', 'importlib.machinery', 'contextlib', 'google', 'google.cloud', 'google.logging', 'mpl_toolkits', 'sphinxcontrib', 'sitecustomize', 'runpy', 'pkgutil', 'weakref', '_weakrefset', 'ipykernel', 'ipykernel._version', 'ipykernel

In [ ]:
import json
json_string = json.dumps(x)
print(json_string)

In [167]:
!pip install pipreqs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [169]:
!pip install session-info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8048 sha256=db0e5e8403684ab18421d122918c7fa00c5c5778a364c711c69cd60ff2e81d1c
  Stored in directory: /root/.cache/pip/wheels/bd/ad/14/6a42359351a18337a8683854cfbba99dd782271f2d1767f87f
Successfully built session-info


In [170]:
!pip install nbconvert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [177]:
pip freeze > requirements.txt

In [179]:
from pip._internal.utils.misc import get_installed_distributions
import sys
#import numpy as np # imported to test whether numpy shows up, which it does!

def get_imported_packages():
    p = get_installed_distributions()
    p = {package.key:package.version for package in p}

    imported_modules = set(sys.modules.keys())
    
    imported_modules.remove('pip')

    modules = [(m, p[m]) for m in imported_modules if p.get(m, False)]

    return modules


def generate_requirements(filepath:str, modules):
    with open(filepath, 'w') as f:
        for module, version in modules:
            f.write(f"{module}=={version}\n")


generate_requirements('newrequirements.txt', get_imported_packages())